# Runs and Stories

At this point, we've built a container class to store the output of `starlab` commands, but we have yet to create a pythonic way to generate those commands in the first place. We're going to do that now. This object (which we will call a `Run`) should contain all of the metadata for a simulation, methods to generate the necessary starlab commands and create `Stories` from them, and references to the resulting `Story` objects. It might be convenient to store `Stories` for all of the transformations that happen prior to integrating, as well.

There are a couple of ways we could approach this task. We could:

1. Make a very generic and flexible system for generating shell commands, or
2. Make specific methods for the commands we will use.

Thinking ahead to building the web frontend, I think option 2 will serve us better, so we'll (more or less) go that route.

Commands fall into three categories:

1. Creation of star clusters
2. Transformation of star clusters
3. Time integration

We'll deal with each of these separately.

## Cluster creation commands

The two cluster creation commands that are most useful are `makeking` and `makeplummer`. Each generates a distribution of stars in space and velocity (effectively, in potential and kinetic energy space) that satisfies a particular model for equilibrium dynamics.  In addition to these, there are a couple of commands that are useful for testing purposes: `makesphere`, and `makecube`. I'd like to support all four of these commands.

Now, for the complexity. Each of these has slightly different arguments; some are mandatory and some are optional. Some take values, and others don't. To summarize:

In [1]:
# creation commands and their arguments
# required arguments include defaults
# optional arguments are split into those that take a value and those that don't
creation_command_arguments = {'makesphere':{'required':{'n':500},
                                            'allowed_value':['s', 'R'],
                                            'allowed_no_value':['i','l','o','u','U']},
                              'makecube':{'required':{'n':500},
                                          'allowed_value':['s', 'L'],
                                          'allowed_no_value':['i','l','o','u']},
                              'makeplummer':{'required':{'n':500},
                                             'allowed_value':['s', 'm', 'r'],
                                             'allowed_no_value':['i','R','o','u']},
                              'makeking':{'required':{'n':500,
                                                      'w':5.0},
                                          'allowed_value':['s', 'b'],
                                          'allowed_no_value':['i','o','u']}}

In [10]:
def build_command(cmd, default_arguments_dict, **cmd_args):
    """Generate a starlab command from arguments, supplying defaults.
    
    Arguments that aren't allowed for a particular command are silently ignored.
    """
    if cmd not in default_arguments_dict.keys():
        raise ValueError('Unrecognized cluster creation command: %s' % cmd)
    command_array = [cmd,]
    args_dict = default_arguments_dict[cmd]
    for arg, default in args_dict['required'].items():
        val = cmd_args.get(arg, default)
        command_array.extend(['-'+arg, val])
    for arg in args_dict['allowed_value']:
        val = cmd_args.get(arg, None)
        if val is not None:
            command_array.extend(['-'+arg, val])
    for arg in args_dict['allowed_no_value']:
        val = cmd_args.get(arg, False)
        if val:
            command_array.append('-'+arg)
            
    return command_array

In [11]:
build_command('makeking', creation_command_arguments, s=1234567, i=3, u=False)

['makeking', '-n', 500, '-w', 5.0, '-s', 1234567, '-i']

This works, but it isn't very tidy. Let me clean things up a bit by using an `Enum` class.

In [46]:
from enum import Enum

class StarlabCommand(Enum):
    """Enumerator class for starlab commands.
    
    Each command takes three parameters:
    
    1. A dictionary of required arguments (with default values),
    2. A list of optional arguments that take a value, and
    3. A list of optional arguments that don't take a value.
    
    If there are parameters which are not, strictly speaking, required
    (i.e., the underlying starlab command will execute without them being
    supplied) but I want to make sure they get into the database, I will
    include them in the required list.
    """
    def __init__(self, required, with_value, without_value):
        """Initialize."""
        
        self.required = required
        self.with_value = with_value
        self.without_value = without_value

    def build_command(self, **cmd_args):
        """Build a command list suitable for passing to subprocess.Run()"""
        
        command_list = [self.name]
        for arg, default in self.required.items():
            val = cmd_args.get(arg, default)
            command_list.extend(['-'+arg, val])
        for arg in self.with_value:
            val = cmd_args.get(arg, None)
            if val is not None:
                command_list.extend(['-'+arg, val])
        for arg in self.without_value:
            val = cmd_args.get(arg, False)
            if val:
                command_list.append('-'+arg)
        return command_list

class StarlabCreationCommand(StarlabCommand):
    """Starlab cluster creation commands.
    """
    makesphere = ({}, ['R'], ['i','l','o','u','U'])
    makecube = ({}, ['L'], ['i','l','o','u'])
    makeplummer = ({}, ['m', 'r'], ['i','R','o','u'])
    makeking = ({'w':5.0}, ['b'], ['i','o','u'])

    def __init__(self, required, with_value, without_value):
        """Initialize.
        
        All creation methods require a number of stars, and I'm adding
        random seed to the required list."""
        self.required = required
        self.with_value = with_value
        self.without_value = without_value

        self.required['n'] = 500
        self.required['s'] = 123456789

In [48]:
StarlabCreationCommand.makeking.build_command(s=987654321, w=1.2, n=2500, i=True)

['makeking', '-s', 987654321, '-n', 2500, '-w', 1.2, '-i']

## Cluster Transformation commands

The two transformation commands we will use in just about every simulation are `makemass` and `scale`; if we're going to have binaries in the simulation we will also use `makesecondary` and `makebinary`.

We can use the same strategy we've already used to generate the commands. In this case, since `scale` doesn't take a random seed, we don't have any required arguments that are common to all of the commands, and we don't need to overload any methods. We do want to keep these as a separate `Enum` class, though, because they're in a different category than the creation commands. 

In [43]:
class StarlabTransformationCommand(StarlabCommand):
    """Starlab cluster transformation commands."""
    makemass = ({'e':-2.35, 'f':1, 's':123456789}, ['h', 'l', 'u'], ['i', 'm'])
    makesecondary = ({'s':123456789}, ['f', 'l', 'm', 'M', 'u'], ['i', 'I', 'q', 'S'])
    scale = ({}, ['e', 'E', 'm', 'q', 'r'], ['c', 's'])
    makebinary = ({'s':123456789}, ['f', 'e', 'l', 'o', 'u'], [])

## Time integration

The integrator has a lot of options, but otherwise, the same strategy works. It seems a little silly to have an `Enum` class with only one item in it, but that's partly a function of the kinds of things we're studying. There are a couple of other integrators included in `starlab`, but they're not designed for our problems of interest.

In [51]:
class StarlabIntegrationCommand(StarlabCommand):
    """Time integration"""
    kira = ({'d':1, 's':123456789, 't':10},
            list('bDefFgGhIkKlLnNqRTWXyzZ'),
            list('aABEioOrSuUvx'))

## Putting it all Together

## Serializing the Run

In [13]:
foo = (None,)

In [14]:
import uuid

In [19]:
int(uuid.uuid1())

303879504568696653710410758444955439609

In [20]:
foo = uuid.uuid1()

In [28]:
foo.fields

(4156668876, 54709, 4581, 179, 213, 57452528112121)

In [29]:
uuid.uuid1().fields

(631299950, 54710, 4581, 179, 213, 57452528112121)

In [33]:
foo.time_mid

54709

In [39]:
print(uuid.uuid4().time_low, uuid.uuid4().time_low)

2502413767 3290443559
